In [ ]:
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po

### Formulation of the (economic) Dispatch problem
* Packages used:
    * Pyomo
    * ...

#### 1. Parameters
* PV-specific:
    * *None*
* BESS-specific:
    * Max Storage Capacity
    * Charging Capacity == Discharge Capacity
    * Charge-Efficiency
    * Discharge-Efficiency
    * *Roundtrip Efficiency
    * Mac SoC
    * Min SoC
* Other system-specific parameters:
    * Inverter efficiency
    * PV-BESS system cost
    * Max Grid Volume
* Other parameters that the dispatcher requires:
    * Profit {calculated from the power flows and electricity price at each time-step *t*}
    * State-of-Charge [SoC] {traced/calculated from the SoC[*t-1*] and power flows to/from BESS}

#### 2. Decision Variables
*Decision variables in the dispatch model are not the same as in the global optimisation problem. In the dispatch problem, the decision variables are the different power flows at each time-step *t*
* PV_to_Load
* PV_to_BESS
* PV_curtailment
* PV_to_Grid
* BESS_to_Load
* BESS_to_Grid
* Grid_to_Load
* Grid_to_BESS

#### 3. Objective Function
* MinimiseCost = 
    $$
    min( 
        \sum_{t=o}^{8760} (
            C_{grid}^i[t] * P_{Grid-to-Load}[t]                                                     %cost of electricity from grid
            + C_{PV+BESS}^i[t] * [P_{PV-to-Load}[t] + P_{BESS-to-Load}[t]]                          %cost of electricity from PV-BESS system
            + C_{penalty}^{Curtailment} * P_{Curtailment}[t]                                        %penalty for curtailing PV production
            - C_{grid}^i[t] * [[P_{BESS-to-Grid}[t] + P_{PV-to-Grid}[t] ] - P_{Grid-to-BESS}[t]]    %profit from energy arbitrage
        )
    )
    $$

#### 4. Constraints
* Load fulfilment
    $$
        P_{Load}[t] = P_{PV-to-Load}[t] + P_{BESS-to-Load}[t] + P_{Grid}[t]
    $$
* Solar production
    $$
        P_{PV production}[t] = \frac{P_{PV-to-Load}[t] + P_{PV-to-BESS}[t] + P_{PV-to-Grid}[t]}{\xi_{inverter}} + P_{Curtailment}[t]
    $$
* SoC range
    $$
        SoC^{min} < SoC[t] < SoC^{max} ; \qquad \forall \ \ t
    $$
* SoC tracking
    $$
        SoC[t] = (
            SoC[t-1]
            + ( P_{PV-to-BESS}[t] + P_{Grid-to-BESS}[t] * \xi_{charge} ) * \Delta{t}
            - \frac{P_{BESS-to-Load}[t] + P_{BESS-to-Grid}[t]}{\xi_{discharge}} * \Delta{t}
        )
    $$
##### 4.1 *Additional* Constraints
* Limiting power flows BESS-to-Grid and PV-to-Grid in energy arbitrage
    $$
        P_{PV-to-Grid}[t] + P_{BESS-to-Grid}[t] <= PowerFlowLimit
    $$
* Limiting the BESS charging to either charge or discharge in the same hour
    $$
        \forall \quad t; \qquad (P_{PV-to-BESS}[t], P_{Grid-to-BESS}[t])*ChargingState\ \ AND\ \ (P_{BESS-to-Load}[t], P_{BESS-to-Grid}[t])*(1-ChargingState)
    $$
    * Subject to:
    * SoC tracking
    $$
        SoC[t] = (
            SoC[t-1]
            + ( P_{PV-to-BESS}[t] + P_{Grid-to-BESS}[t] * \xi_{charge} * ChargingState) * \Delta{t}
            - \frac{P_{BESS-to-Load}[t] + P_{BESS-to-Grid}[t]}{\xi_{discharge}} * (1-ChargingState) * \Delta{t}
        )
    $$

___
### Implementation notes
* General approach (for now) is to use the *ConcreteModel*
    * If *AbstractModel*, how could the model parameters,variables,etc be populated from the datafile OR the inputs of the model
    * *Why?* --> In the future, the model could be adapted to be compiled and to describe different kinds of and setups of energy systems with different technologies
* Be mindful of how the data is loaded and called in the model to avoid integration errors (like the previous error with numpy)
* Convert the model to kW (previously MW) to avoid issues/error with integer to float conversion
* *OBS*: SoC[*t*] needs a previous value, so we need to artificially override the initial SoC (or implement an SoC for *t* = -1)
* In future implementations/improvements, try converting the time-step to seconds/minutes.
    * *Reason* is to have a more realistic dispatch profile where BESS can charge and discharge in the same hour
* To keep in mind for validation
    * I could also run it with different curtailment penalties and see their effect
    * Underlying difference in optimisation problem when optimising only each time-step vs. optimising a time series window

The problem will be defined as a worker/tasks set, where the power flows are different "workers" and hours in day are differet "tasks"
The values in each cell has to be the cost, so I will eventually have to make some sort of a generator to populate it quickly

| Flow \ Hour |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  8 |  9 | 10 | ... | 48 | H
|:---------------:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:
| P_PV-to-BESS      | $$p_{f, h}$$ |  $$p_{f, h}$$ |  $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | ... | - |
| P_PV-to-Load      | $$p_{f, h}$$ |  $$p_{f, h}$$ |  $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | ... | - |
| P_BESS-to-Load    | $$p_{f, h}$$ |  $$p_{f, h}$$ |  $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | ... | - |
| ...               | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | - |
| P_n               | - |  - |  - |  - | - |  - | - | - | - | - | - | - |
**F**

Subject to:
$$
minimize[
    \quad
    \sum_{h=1}^{H={1}}\ (
        \quad
        \sum_{f=1}^{F}\ (\ P_{f, h}\  *\  Cost_{f, h}\ )
        \quad
    )
    \
]
$$
or shorthened for 1 hour interval:
| Flow \ Hour |  1 |
|:---------------:|---:|
| P_PV-to-BESS      | $$p_{f_1}$$ |
| P_PV-to-Load      | $$p_{f_2}$$ |
| P_BESS-to-Load    | $$p_{f_3}$$ |
| ...               | ... |
| P_n               | $$p_{f_n}$$ |
**F**
$$
minimize
    \ \sum_{f=1}^{F}\ (\ P_{f}\ *\ Cost_{f}\ )

$$

In [ ]:
## Primary data parameters of our scenarios
pv_price= 80                #https://data.nrel.gov/submissions/53 in EUR/kW
bess_price= 200             #https://doi.org/10.1016/j.solener.2018.08.061 in EUR/kWh, adjusted for price decreases
pv_opex= 17                 #EUR/kWh ->reference in excel
bess_opex= 0.125            #EUR/kWh ->reference in excel

pv_co2= 33                  #kgCO2eq/kW_powerDC ->reference in excel
bess_co2= 100               #kgCO2eq/kWh_capacity ->reference in excel
pv_opex_co2= 0              #kgCO2eq/kW_powerDC ->assumption
bess_opex_co2= 0            #kgCO2eq/kW_powerDC ->assumption
discount_rate= 0.0485       #assumption
lifetime_project= 32        #for the project lifetime
lifetime_bess= 8            #for the BESS lifetime
degradation_rate= 0.025     #assumption (based on reaching 80% SoH in 8 years)

params = {
    'pv_price':         pv_price,
    'bess_price':       bess_price,
    'pv_opex':          pv_opex,
    'bess_opex':        bess_opex,
    'pv_co2':           pv_co2,
    'bess_co2':         bess_co2,
    'pv_opex_co2':      pv_opex_co2,
    'bess_opex_co2':    bess_opex_co2,
    'discount_rate':    discount_rate,
    'lifetime_project': lifetime_project,
    'lifetime_bess':    lifetime_bess,
    'degradation_rate': degradation_rate
}

In [ ]:
p_solar=    0.01                # 1MW = 1000kW
p_bess=     5000                # 5MW = 5000kW
t_bess=     4                   # 4 hour BESS duration at nominal power
e_bess=     p_bess * t_bess     # determining the BESS size in kWh
e_bess

In [ ]:
hour= 1

'''
Convert flows to numerical values:
1-3 ->  flows to Load
1   P_PV_to_Load,
2   P_BESS_to_Load,
3   P_Grid_to_Load,

4-5 ->  flows to BESS
4   P_PV_to_BESS,
5   P_Grid_to_BESS,

6-7 ->  flows to Grid
6   P_PV_to_Grid,
7   P_BESS_to_Grid,

8   ->  Curtailment
8   P_PV_curtailment,
'''
flows=  [
    'P_PV_to_Load',
    'P_PV_to_BESS',
    'P_PV_curtailment',
    'P_PV_to_Grid',
    'P_BESS_to_Load',
    'P_BESS_to_Grid',
    'P_Grid_to_Load',
    'P_Grid_to_BESS',
]
costs= {
    'P_PV_to_Load'      : 3/1000,       #Cost to send PV to Load
    'P_PV_to_BESS'      : 3/1000,       #Cost to charge from PV
    'P_PV_to_Grid'      : (24-3)/1000,  #Cost to send PV to Grid
    'P_PV_curtailment'  : 1000/1000,    #Cost/Penalty of Curtailment
    'P_BESS_to_Load'    : 6/1000,       #Cost to discharge BESS to Load
    'P_BESS_to_Grid'    : (24-6)/1000,  #Cost to discharge BESS to Grid
    'P_Grid_to_Load'    : 24/1000,      #Cost to supply Load from the grid
    'P_Grid_to_BESS'    : 24/1000,      #Cost to charge BESS from the grid
}

demand=             3000
pv_production=      3366 * p_solar
grid_production=    1000000
costs

In [ ]:
model= pe.ConcreteModel()

In [ ]:
#model.flows=    pe.Set(initialize= flows)

In [ ]:
soc_initial=            int(e_bess * 0.5)
efficiency_charge=      0.98
efficiency_discharge=   0.96
efficiency_inverter=    0.97

In [ ]:
model.grid_production=      pe.Param(initialize= grid_production)
model.demand=               pe.Param(initialize= demand)
model.pv_production=        pe.Param(initialize= pv_production)
#model.costs=                pe.Param(model.flows, initialize= costs, default= 1000)
model.SoC_initial=          pe.Param(initialize= soc_initial)
model.Efficiency_inverter=  pe.Param(initialize= efficiency_inverter)
model.Efficiency_charge=    pe.Param(initialize= efficiency_charge)
model.Efficiency_discharge= pe.Param(initialize= efficiency_discharge)
model.SoC=                  pe.Param(initialize= soc_initial)

In [ ]:
print("Initialized Parameters:")
for param_name, param_obj in model.component_map(pe.Param, active=True).items():
    if param_name == 'SoC_initial':
        #for flow in model.flows:
        #    print(f"{param_name}: {param_obj[flow]}")
        print(f"{param_name}: {param_obj.value}")

In [ ]:
#model.p=    pe.Var(model.flows, domain= pe.Reals, bounds= (0, 1000000))
model.P_PV_to_Load=     pe.Var(domain= pe.Reals, bounds= (0, 1000000))
model.P_PV_to_BESS=     pe.Var(domain= pe.Reals, bounds= (0, 1000000))
model.P_PV_curtailment= pe.Var(domain= pe.Reals, bounds= (0, 1000000))
model.P_PV_to_Grid=     pe.Var(domain= pe.Reals, bounds= (0, 1000000))
model.P_BESS_to_Load=   pe.Var(domain= pe.Reals, bounds= (0, 1000000))
model.P_BESS_to_Grid=   pe.Var(domain= pe.Reals, bounds= (0, 1000000))
model.P_Grid_to_Load=   pe.Var(domain= pe.Reals, bounds= (0, 1000000))
model.P_Grid_to_BESS=   pe.Var(domain= pe.Reals, bounds= (0, 1000000))

In [ ]:
model.display()

In [ ]:
minCosts= (
    model.P_PV_to_Load * costs['P_PV_to_Load']
    + model.P_BESS_to_Load * costs['P_BESS_to_Load']
    + model.P_Grid_to_Load * costs['P_Grid_to_Load']
    + model.P_PV_curtailment * costs['P_PV_curtailment']
    + model.P_Grid_to_BESS * costs['P_Grid_to_BESS']
    - model.P_PV_to_Grid * costs['P_PV_to_Grid']
    - model.P_BESS_to_Grid * costs['P_BESS_to_Grid']
)
model.objective=    pe.Objective(
    sense=  pe.minimize,
    expr=   minCosts
)

In [ ]:
model.display()

In [ ]:
model.pprint()

In [ ]:
'''
1-3 ->  flows to Load
1   P_PV_to_Load
2   P_BESS_to_Load
3   P_Grid_to_Load

4-5 ->  flows to BESS
4   P_PV_to_BESS
5   P_Grid_to_BESS

6-7 ->  flows to Grid
6   P_PV_to_Grid
7   P_BESS_to_Grid

8   ->  Curtailment
8   P_PV_curtailment
'''
loadFullfilment=    model.P_PV_to_Load + model.P_BESS_to_Load + model.P_Grid_to_Load == demand
pvProduction=       (model.P_PV_to_Load + model.P_PV_to_BESS + model.P_PV_to_Grid)/model.Efficiency_inverter + model.P_PV_curtailment == pv_production
#socRangeMin=        (e_bess * 0.1) <= model.SoC
#socRangeMax=        model.SoC <= (e_bess * 0.9)
arbitrageFlow=      (model.P_BESS_to_Grid + model.P_PV_to_Grid) <= (p_solar * 1000000 + p_bess)
#socTracker=         model.SoC == model.SoC_initial + (model.P_PV_to_BESS + model.P_Grid_to_BESS) * model.Efficiency_charge - (model.P_BESS_to_Load + model.P_BESS_to_Grid)/ model.Efficiency_discharge
#socTracker=         model.SoC_initial.value >= (((model.P_PV_to_BESS + model.P_Grid_to_BESS) * model.Efficiency_charge) * model.Efficiency_inverter - ((model.P_BESS_to_Load + model.P_BESS_to_Grid)/ model.Efficiency_discharge) * model.Efficiency_inverter)
socTrackerSimple=   (model.P_PV_to_BESS + model.P_Grid_to_BESS - model.P_BESS_to_Load - model.P_BESS_to_Grid) <= model.SoC_initial

In [ ]:
model.fullfilLoad=      pe.Constraint(expr= loadFullfilment)
model.allocatePV=       pe.Constraint(expr= pvProduction)
model.arbitrageLimit=   pe.Constraint(expr= arbitrageFlow)
#model.socRangeMin=     pe.Constraint(expr= socRangeMin)
#model.socRangeMax=     pe.Constraint(expr= socRangeMax)
model.socTracking=      pe.Constraint(expr= socTrackerSimple)

In [ ]:
model.display()

In [ ]:
solver=     pe.SolverFactory('glpk')
results=    solver.solve(
    model,
    #tee= True
)
results.solver.status

In [ ]:
results

In [ ]:
print(pe.value(model.objective))

In [ ]:
for var in model.component_data_objects(pe.Var, active=True):
    print('------------------------------------------------------------')
    print(f"{var.name}: {pe.value(var)}")